# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 19 2022 8:30AM,248905,10,SNAP1018221,"Snap Medical Industries, LLC",Released
1,Oct 19 2022 8:30AM,248905,10,SNAP1018222,"Snap Medical Industries, LLC",Released
2,Oct 19 2022 8:30AM,248905,10,SNAPSC42443704,"Snap Medical Industries, LLC",Released
3,Oct 19 2022 8:30AM,248909,10,Enova-10826,Emerginnova,Released
4,Oct 19 2022 8:30AM,248909,10,Enova-10827,Emerginnova,Released
5,Oct 19 2022 8:30AM,248913,10,MSP214785-BO,Methapharm-G,Released
6,Oct 19 2022 8:30AM,248913,10,MSP214836-BO,Methapharm-G,Released
7,Oct 18 2022 4:19PM,248915,19,ACG-2102491599,ACG North America LLC,Released
8,Oct 18 2022 4:19PM,248915,19,ACG-2102491600,ACG North America LLC,Released
9,Oct 18 2022 3:28PM,248904,20,8629010-BO,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,248904,Released,1
28,248905,Released,3
29,248909,Released,2
30,248913,Released,2
31,248915,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248904,NaN,NaN,1.0
248905,NaN,NaN,3.0
248909,NaN,NaN,2.0
248913,NaN,NaN,2.0
248915,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248836,40.0,2.0,3.0
248837,0.0,25.0,12.0
248850,0.0,0.0,1.0
248851,0.0,0.0,2.0
248861,5.0,14.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248836,40,2,3
248837,0,25,12
248850,0,0,1
248851,0,0,2
248861,5,14,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248836,40,2,3
1,248837,0,25,12
2,248850,0,0,1
3,248851,0,0,2
4,248861,5,14,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248836,40,2,3
1,248837,,25,12
2,248850,,,1
3,248851,,,2
4,248861,5,14,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 19 2022 8:30AM,248905,10,"Snap Medical Industries, LLC"
3,Oct 19 2022 8:30AM,248909,10,Emerginnova
5,Oct 19 2022 8:30AM,248913,10,Methapharm-G
7,Oct 18 2022 4:19PM,248915,19,ACG North America LLC
9,Oct 18 2022 3:28PM,248904,20,"Exact-Rx, Inc."
10,Oct 18 2022 3:27PM,248903,22,"NBTY Global, Inc."
11,Oct 18 2022 3:20PM,248902,20,"ACI Healthcare USA, Inc."
12,Oct 18 2022 2:38PM,248893,10,Emerginnova
14,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc."
17,Oct 18 2022 2:19PM,248889,10,Methapharm-G


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 19 2022 8:30AM,248905,10,"Snap Medical Industries, LLC",,,3
1,Oct 19 2022 8:30AM,248909,10,Emerginnova,,,2
2,Oct 19 2022 8:30AM,248913,10,Methapharm-G,,,2
3,Oct 18 2022 4:19PM,248915,19,ACG North America LLC,,,2
4,Oct 18 2022 3:28PM,248904,20,"Exact-Rx, Inc.",,,1
5,Oct 18 2022 3:27PM,248903,22,"NBTY Global, Inc.",,,1
6,Oct 18 2022 3:20PM,248902,20,"ACI Healthcare USA, Inc.",,,1
7,Oct 18 2022 2:38PM,248893,10,Emerginnova,,,2
8,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc.",,5,1
9,Oct 18 2022 2:19PM,248889,10,Methapharm-G,,5,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 19 2022 8:30AM,248905,10,"Snap Medical Industries, LLC",3,,
1,Oct 19 2022 8:30AM,248909,10,Emerginnova,2,,
2,Oct 19 2022 8:30AM,248913,10,Methapharm-G,2,,
3,Oct 18 2022 4:19PM,248915,19,ACG North America LLC,2,,
4,Oct 18 2022 3:28PM,248904,20,"Exact-Rx, Inc.",1,,
5,Oct 18 2022 3:27PM,248903,22,"NBTY Global, Inc.",1,,
6,Oct 18 2022 3:20PM,248902,20,"ACI Healthcare USA, Inc.",1,,
7,Oct 18 2022 2:38PM,248893,10,Emerginnova,2,,
8,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc.",1,5,
9,Oct 18 2022 2:19PM,248889,10,Methapharm-G,1,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 19 2022 8:30AM,248905,10,"Snap Medical Industries, LLC",3,,
1,Oct 19 2022 8:30AM,248909,10,Emerginnova,2,,
2,Oct 19 2022 8:30AM,248913,10,Methapharm-G,2,,
3,Oct 18 2022 4:19PM,248915,19,ACG North America LLC,2,,
4,Oct 18 2022 3:28PM,248904,20,"Exact-Rx, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 19 2022 8:30AM,248905,10,"Snap Medical Industries, LLC",3.0,NaN,NaN
1,Oct 19 2022 8:30AM,248909,10,Emerginnova,2.0,NaN,NaN
2,Oct 19 2022 8:30AM,248913,10,Methapharm-G,2.0,NaN,NaN
3,Oct 18 2022 4:19PM,248915,19,ACG North America LLC,2.0,NaN,NaN
4,Oct 18 2022 3:28PM,248904,20,"Exact-Rx, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 19 2022 8:30AM,248905,10,"Snap Medical Industries, LLC",3.0,0.0,0.0
1,Oct 19 2022 8:30AM,248909,10,Emerginnova,2.0,0.0,0.0
2,Oct 19 2022 8:30AM,248913,10,Methapharm-G,2.0,0.0,0.0
3,Oct 18 2022 4:19PM,248915,19,ACG North America LLC,2.0,0.0,0.0
4,Oct 18 2022 3:28PM,248904,20,"Exact-Rx, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2737779,17.0,18.0,4.0
15,497673,15.0,27.0,40.0
19,1493210,14.0,14.0,5.0
20,497806,2.0,0.0,0.0
22,995507,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2737779,17.0,18.0,4.0
1,15,497673,15.0,27.0,40.0
2,19,1493210,14.0,14.0,5.0
3,20,497806,2.0,0.0,0.0
4,22,995507,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,17.0,18.0,4.0
1,15,15.0,27.0,40.0
2,19,14.0,14.0,5.0
3,20,2.0,0.0,0.0
4,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,17.0
1,15,Released,15.0
2,19,Released,14.0
3,20,Released,2.0
4,22,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,22
Status,,,,,
Completed,4.0,40.0,5.0,0.0,0.0
Executing,18.0,27.0,14.0,0.0,0.0
Released,17.0,15.0,14.0,2.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,22
0,Completed,4.0,40.0,5.0,0.0,0.0
1,Executing,18.0,27.0,14.0,0.0,0.0
2,Released,17.0,15.0,14.0,2.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,22
0,Completed,4.0,40.0,5.0,0.0,0.0
1,Executing,18.0,27.0,14.0,0.0,0.0
2,Released,17.0,15.0,14.0,2.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()